In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key=os.getenv('GROQ_API_KEY')

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)


In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi,My name is Niruppa and I am a Genai enthusiast")])

AIMessage(content="Hi Niruppa! It's great to meet you.\n\nI'm happy to hear you're a GenAI enthusiast! What aspects of generative AI are you most interested in? \n\nDo you have any specific projects you're working on or questions you'd like to discuss? I'm always eager to learn more about how people are using and exploring the potential of this technology.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 23, 'total_tokens': 107, 'completion_time': 0.152727273, 'prompt_time': 0.00132835, 'queue_time': 0.25663950900000004, 'total_time': 0.154055623}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--88016287-02e4-4a1f-b851-e9850ed87062-0', usage_metadata={'input_tokens': 23, 'output_tokens': 84, 'total_tokens': 107})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi,My name is Niruppa and I am a Genai enthusiast"),
        AIMessage(content="Hi Niruppa,\n\nIt's nice to meet you! I'm glad to hear you're a GenAI enthusiast. \n\nWhat aspects of generative AI are you most interested in? I'm always happy to chat about the latest developments, applications, or even ethical considerations surrounding this exciting field.\n"),
        HumanMessage(content="Hey whats my name and what do I like?")
    ]

)

AIMessage(content="As an AI, I have no memory of past conversations and don't have access to any personal information about you, including your name or your likes. \n\nYou introduced yourself as Niruppa earlier in our conversation!  \n\nWhat are some things you enjoy? I'd love to learn more about you. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 107, 'total_tokens': 176, 'completion_time': 0.125454545, 'prompt_time': 0.00351189, 'queue_time': 0.25373704999999996, 'total_time': 0.128966435}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--96106b3c-e49b-4d49-883a-0162b655e96a-0', usage_metadata={'input_tokens': 107, 'output_tokens': 69, 'total_tokens': 176})

Message history is essential for maintaining context across different sessions and users. In real-world applications, multiple sessions may occur simultaneously, and each session must retain its own context. This is managed using session IDs and message history.

In [7]:
# message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

Implementing Session-Based Message History
We create a function to manage session-based message history. Each session ID corresponds to a separate chat history, stored in a dictionary.

In [8]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

Using RunnableWithMessageHistory for Stateful Chains
We wrap the model with RunnableWithMessageHistory, providing the model and the session history function. This allows the chatbot to maintain state across interactions for each session.

In [9]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

Configuring and Using Session IDs
We define a configuration with a session ID and use it to interact with the chatbot. The session ID ensures that the chatbot remembers the context for that particular session.

In [10]:
from langchain_core.messages import HumanMessage

# ✅ Pass session_id inside config['configurable']
config = {
    "configurable": {
        "session_id": "chat1"
    }
}

# ✅ Pass a list of HumanMessage as input
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Niruppa and I am a GenAI enthusiast.")],
    config=config
)

print(response.content)


Hi Niruppa, it's nice to meet you! 

I'm glad to hear you're a GenAI enthusiast. It's an exciting field with a lot of potential. 

What are you most interested in about GenAI? Are you working on any projects yourself? I'd love to hear more about your passions!



In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

print(response.content)

Your name is Niruppa.  

I remember that from our introduction! 😊  Is there anything else I can help you with?



In [12]:
config1 = {
    "configurable": {
        "session_id": "chat2"
    }
}
response = with_message_history.invoke([HumanMessage(content="What's my name?")], config=config1)
print(response.content)

As an AI, I have no memory of past conversations and don't know your name. If you'd like to tell me, I'd be happy to know!



In [13]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is john")],
    config=config
)

print(response.content)

Hi John! 

It's nice to meet you too. 

How can I help you today? 🙂



In [14]:
# Ask again in the same session
response = with_message_history.invoke([HumanMessage(content="What's my name?")], config=config1)
print(response.content)

Since I don't have access to past conversations, I don't know your name.  

Would you like to tell me? 😊  



Using Prompt Templates --
Prompt templates help turn raw user information into a format that a language model can work with. If the raw user input is just a message, we can use prompt templates to structure it before passing it to the language model.

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is niruppa")]})

AIMessage(content="Hi Niruppa! It's nice to meet you. 😊 \n\nI'm ready to answer your questions. What can I help you with today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 31, 'total_tokens': 68, 'completion_time': 0.067272727, 'prompt_time': 0.0014781, 'queue_time': 0.25422582, 'total_time': 0.068750827}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--226cb014-6390-4f5c-82c4-d04af270c63a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 37, 'total_tokens': 68})

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [20]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Niruppa")],
    config=config
)

In [21]:
response

AIMessage(content="Hi Niruppa!  It's nice to meet you too.  \n\nWhat can I do for you today? 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 78, 'total_tokens': 107, 'completion_time': 0.052727273, 'prompt_time': 0.00235867, 'queue_time': 0.25682536, 'total_time': 0.055085943}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--71afb01a-cbcb-45f6-8196-5c8d3d97b0b8-0', usage_metadata={'input_tokens': 78, 'output_tokens': 29, 'total_tokens': 107})

In [22]:
# more complex

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.Answer all questions to the best of your ability in {language}."

        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Niruppa")],"language":"German"})
response.content

'Hallo Niruppa, schön dich kennenzulernen!  Wie kann ich dir heute helfen? 😊\n'

In [25]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [26]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke({
    'messages':[HumanMessage(content="Hi My name is Niruppa")],"language":"German"},
    config=config
)
    
response.content

'Hallo Niruppa, schön dich kennenzulernen! 👋 \n\nWie kann ich dir heute helfen?  \n'

In [27]:
response=with_message_history.invoke({
    'messages':[HumanMessage(content="Whats my name")],"language":"German"},
    config=config
)
    
response.content

'Dein Name ist Niruppa. 😊 \n'

# Managing the Chat Conversation History Using Langchain
Proper management of conversation history is crucial, as an unmanaged list of messages can grow unbounded and potentially overflow the context window of the language model (LM). Therefore, it is important to add a step that limits the size of the messages being passed.

Understanding trim_messages--
The trim_messages helper is used to reduce the number of messages sent to the model. It allows specifying how many tokens to keep and provides other parameters such as whether to always keep the system message and whether to allow partial messages.

In [32]:
from langchain_core.messages import SystemMessage
from langchain_core.messages import trim_messages

trimmer=trim_messages(
    max_tokens=70, # limit of tokens
    strategy="last", # from the last conversation count the token
    token_counter=model, # function for counting tokens
    include_system=True, # system message include
    allow_partial=False, # no partial information
    start_on="human" # start from human conversation
)

messages = [
    SystemMessage(content="You are a good assistant."),
    HumanMessage(content="Hey, I am Bob."),
    AIMessage(content="Hi."),
    HumanMessage(content="I like vanilla ice cream."),
    AIMessage(content="Nice."),
    HumanMessage(content="What is two plus two?"),
    AIMessage(content="Four."),
    HumanMessage(content="Thanks.")
]

trimmed = trimmer.invoke(messages)

In [33]:
trimmed

[SystemMessage(content='You are a good assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hey, I am Bob.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is two plus two?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Four.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks.', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain =(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt |model
)
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like ")],
    "language":"English"
    }
)
response.content

'You said you like vanilla ice cream. 😊  \n'

In [35]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did i ask ")],
    "language":"English"
    }
)
response.content

'You asked "What is two plus two?" \n\n\n\nLet me know if you have any other questions!\n'

In [36]:
# wrap in message history
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")
config = {"configurable":{"session_id":"chat5"}}

In [37]:
response=with_message_history.invoke({
    'messages':[HumanMessage(content="Whats my name")],"language":"German"},
    config=config
)
    
response.content

'Ich weiß nicht, wie du heißt. Könntest du mir deinen Namen sagen? \n\n'